In [28]:
import pandas as pd
import re

In [29]:
df = pd.read_excel(r"C:\Users\u326421\Desktop\IHG - Text\gcp_op1.xlsx")

In [30]:
df.columns

Index(['ID_SID', 'Text', 'level_2', 'ent_op', 'entity', 'type', 'salience'], dtype='object')

In [31]:
df["Text"] = df["Text"].fillna(" ")

In [32]:
df['Text'] = df['Text'].apply(lambda x: re.sub('[^.,a-zA-Z0-9 \n\.]', '',x))

In [33]:
df['Text'] = df['Text'].str.lower()

In [34]:
df.head()

,ID_SID,Text,level_2,ent_op,entity,type,salience
0,17504_1000_Q1_1,close to clinic and could get shuttle,0,"('clinic', 'LOCATION', 0.5232729315757751)",clinic,LOCATION,0.523273
1,17504_1000_Q1_1,close to clinic and could get shuttle,1,"('shuttle', 'OTHER', 0.47672703862190247)",shuttle,OTHER,0.476727
2,17504_1000_Q2_1,i booked a handicapped room,0,"('room', 'LOCATION', 1.0)",room,LOCATION,1.000000
3,17504_1000_Q2_10,came back home because we couldnt afford ano...,0,"('home', 'LOCATION', 1.0)",home,LOCATION,1.000000
4,17504_1000_Q2_11,was told i should call the day before so the...,0,"('room', 'LOCATION', 0.8343371152877808)",room,LOCATION,0.834337


In [35]:
df2 = df

In [9]:
# df2  = df.set_index('Unique_ID').Text.str.split(".").apply(pd.Series).stack().reset_index(name = 'Text')

In [10]:
# df2['Sentence_ID'] = df.set_index('Unique_ID').Text.str.split(".").apply(pd.Series).stack().reset_index(name = 'Text').\
# apply(lambda x : str(x['Unique_ID']) + "_" + str(x['level_1'] + 1), axis = 1)
# df2.head()

,Unique_ID,level_1,Text,Sentence_ID
0,2301_1_Q1,0,location and friendly professional staff,2301_1_Q1_1
1,2301_1_Q1,1,valet service was a plus,2301_1_Q1_2
2,2301_2_Q1,0,it was great the only thing that slightly bugg...,2301_2_Q1_1
3,2301_2_Q1,1,i work across the lot from a marriott and the...,2301_2_Q1_2
4,2301_3_Q1,0,clean and very friendly staff,2301_3_Q1_1


In [11]:
# df2 = df2.drop(["Unique_ID" , "level_1"],  axis= 1)

In [14]:
# df2.dropna().shape

(10550, 2)

In [467]:
# import wordninja

In [468]:
# wordninja.split('derekanderson')

['derek', 'anderson']

In [469]:
# def ninjasplit(x):
#     l1 =[]
#     for str in x:
#         l1.append(wordninja.split(x))
#     return l1

In [470]:
# df2['wn_sent'] = df2.Text.apply(lambda x: " ".join(wordninja.split(x)))

### **GETTING ENTITIES**

In [36]:
from textblob import TextBlob
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\u326421\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\u326421\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\u326421\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [37]:
#Finding nouns to get entities
df2['nouns'] = df2.Text.apply(lambda x : [a for a,b in TextBlob(x).pos_tags if b in ['NN','NNS', 'NNP', 'NNPS']])

In [38]:
df2['nouns'] = df2['nouns'].apply(lambda x: ",".join(x))

In [39]:
df2.head(2)

,ID_SID,Text,level_2,ent_op,entity,type,salience,nouns
0,17504_1000_Q1_1,close to clinic and could get shuttle,0,"('clinic', 'LOCATION', 0.5232729315757751)",clinic,LOCATION,0.523273,clinic
1,17504_1000_Q1_1,close to clinic and could get shuttle,1,"('shuttle', 'OTHER', 0.47672703862190247)",shuttle,OTHER,0.476727,clinic


In [40]:
#Making sure each row has only one entity
df1  = df2.set_index('ID_SID').nouns.str.split(",").apply(pd.Series).stack().reset_index(name = 'nouns')

In [41]:
##Making sure each row has only one entity
df1['ID'] = df2.set_index('ID_SID').nouns.str.split(",").apply(pd.Series).stack().reset_index(name = 'nouns').\
apply(lambda x : str(x['ID_SID']) + "_" + str(x['level_1'] + 1), axis = 1)
df1.head()

,ID_SID,level_1,nouns,ID
0,17504_1000_Q1_1,0,clinic,17504_1000_Q1_1_1
1,17504_1000_Q1_1,0,clinic,17504_1000_Q1_1_1
2,17504_1000_Q2_1,0,i,17504_1000_Q2_1_1
3,17504_1000_Q2_1,1,room,17504_1000_Q2_1_2
4,17504_1000_Q2_10,0,home,17504_1000_Q2_10_1


In [42]:
df1 = df1.drop(["level_1"],  axis= 1)

In [43]:
df1 = pd.merge(df1, df2, on='ID_SID')

In [44]:
df1 = df1.drop(['nouns_y'],  axis= 1)

In [45]:
df1 = df1.rename(columns={"nouns_x": "nouns"})

In [46]:
df1

,ID_SID,nouns,ID,Text,level_2,ent_op,entity,type,salience
0,17504_1000_Q1_1,clinic,17504_1000_Q1_1_1,close to clinic and could get shuttle,0,"('clinic', 'LOCATION', 0.5232729315757751)",clinic,LOCATION,0.523273
1,17504_1000_Q1_1,clinic,17504_1000_Q1_1_1,close to clinic and could get shuttle,1,"('shuttle', 'OTHER', 0.47672703862190247)",shuttle,OTHER,0.476727
2,17504_1000_Q1_1,clinic,17504_1000_Q1_1_1,close to clinic and could get shuttle,0,"('clinic', 'LOCATION', 0.5232729315757751)",clinic,LOCATION,0.523273
3,17504_1000_Q1_1,clinic,17504_1000_Q1_1_1,close to clinic and could get shuttle,1,"('shuttle', 'OTHER', 0.47672703862190247)",shuttle,OTHER,0.476727
4,17504_1000_Q2_1,i,17504_1000_Q2_1_1,i booked a handicapped room,0,"('room', 'LOCATION', 1.0)",room,LOCATION,1.000000
...,...,...,...,...,...,...,...,...,...
518315,2301_9_Q1_1,staff,2301_9_Q1_1_2,politeness of staff,1,"('staff', 'PERSON', 0.11625086516141891)",staff,PERSON,0.116251
518316,2301_9_Q1_1,politeness,2301_9_Q1_1_1,politeness of staff,0,"('politeness', 'OTHER', 0.8837491273880005)",politeness,OTHER,0.883749
518317,2301_9_Q1_1,politeness,2301_9_Q1_1_1,politeness of staff,1,"('staff', 'PERSON', 0.11625086516141891)",staff,PERSON,0.116251
518318,2301_9_Q1_1,staff,2301_9_Q1_1_2,politeness of staff,0,"('politeness', 'OTHER', 0.8837491273880005)",politeness,OTHER,0.883749


In [47]:
# import spacy
# nlp = spacy.load("en_core_web_sm")

In [48]:
# def enttags(text):
#     doc = nlp(text)
#     l1 =[]
#     for ent in doc.ents:
#         l1.append((ent.text,ent.label_))
#     return l1

In [49]:
# enttags("Apple is looking at buying U.K. startup for $1 billion")

In [50]:
# df1["Spacy_Entity"] = df1.Text.map(str).apply(lambda x: enttags(x))

In [51]:
# df1.head(5)

In [52]:
# df1.Spacy_Entity.apply(lambda x: x.replace("[","")).\
# apply(lambda x: x.replace("]","")

In [53]:
df1.head()

,ID_SID,nouns,ID,Text,level_2,ent_op,entity,type,salience
0,17504_1000_Q1_1,clinic,17504_1000_Q1_1_1,close to clinic and could get shuttle,0,"('clinic', 'LOCATION', 0.5232729315757751)",clinic,LOCATION,0.523273
1,17504_1000_Q1_1,clinic,17504_1000_Q1_1_1,close to clinic and could get shuttle,1,"('shuttle', 'OTHER', 0.47672703862190247)",shuttle,OTHER,0.476727
2,17504_1000_Q1_1,clinic,17504_1000_Q1_1_1,close to clinic and could get shuttle,0,"('clinic', 'LOCATION', 0.5232729315757751)",clinic,LOCATION,0.523273
3,17504_1000_Q1_1,clinic,17504_1000_Q1_1_1,close to clinic and could get shuttle,1,"('shuttle', 'OTHER', 0.47672703862190247)",shuttle,OTHER,0.476727
4,17504_1000_Q2_1,i,17504_1000_Q2_1_1,i booked a handicapped room,0,"('room', 'LOCATION', 1.0)",room,LOCATION,1.000000


### **GETTING DESCRIPTORS**

In [ ]:
#Finding adjectives for descriptors
df1['textblob_adjtags'] = df1.Text.apply(lambda x : [a for a,b in TextBlob(x).pos_tags if b in ['JJ','JJR','JJS']])

In [ ]:
df1['textblob_adjtags'] = df1['textblob_adjtags'].apply(lambda x: ",".join(x))

In [ ]:
#Finding adverbs for descriptors
df1['textblob_advtags'] = df1.Text.apply(lambda x : [a for a,b in TextBlob(x).pos_tags if b in ['RB','RBR','RBS']])

In [ ]:
df1['textblob_advtags'] = df1['textblob_advtags'].apply(lambda x: ",".join(x))

In [ ]:
df1.head()

In [492]:
#For descriptors
#Function to remove stop words
from spacy.lang.en import English
nlp = English()

def rem_stopwords(text):
  my_doc = nlp(text)
  # Create list of word tokens
  token_list = []
  for token in my_doc:
      token_list.append(token.text)

  from spacy.lang.en.stop_words import STOP_WORDS

  # Create list of word tokens after removing stopwords
  filtered_sentence =[] 

  for word in token_list:
      lexeme = nlp.vocab[word]
      if lexeme.is_stop == False:
        filtered_sentence.append(word) 

  filtered_sentence = " ".join(filtered_sentence)
  return filtered_sentence

In [493]:
df1['adjtags_removedSW'] = df1.Text.apply(rem_stopwords).apply(lambda x : ",".join([a for a,b in TextBlob(x).pos_tags if b in ['JJ','JJR','JJS']])) 
df1.head(5)

,nouns,ID,Text,wn_sent,Spacy_Entity,textblob_adjtags,textblob_advtags,adjtags_removedSW
0,location,2301_1_Q1_1_1,location and friendly professional staff,location and friendly professional staff,[],"friendly,professional",,"friendly,professional"
1,staff,2301_1_Q1_1_2,location and friendly professional staff,location and friendly professional staff,[],"friendly,professional",,"friendly,professional"
2,valet,2301_1_Q1_2_1,valet service was a plus,valet service was a plus,[],,,
3,service,2301_1_Q1_2_2,valet service was a plus,valet service was a plus,[],,,
4,plus,2301_1_Q1_2_3,valet service was a plus,valet service was a plus,[],,,


In [494]:
#Function to get Stop words using NLTK
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
nltk.download('stopwords')

set(stopwords.words('english'))

def rem_stopwords_stemming(text):
  stop_words = set(stopwords.words('english')) 
  
  word_tokens = word_tokenize(text) 
  filtered_sentence = [] 
  
  for w in word_tokens: 
      if w not in stop_words: 
          filtered_sentence.append(w) 

  Stem_words = []
  ps =PorterStemmer()
  for w in filtered_sentence:
      rootWord=ps.stem(w)
      Stem_words.append(rootWord)
  Stem_words = " ".join(Stem_words)

  return Stem_words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\u326421\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [495]:
# Lemmatization using spacy - does not remove stop words

import en_core_web_sm
nlp = en_core_web_sm.load()

def lemma_spacy(doc):
  doc = nlp(doc)
  lemma_word1 = [] 
  for token in doc:
      lemma_word1.append(token.lemma_)
  return " ".join(lemma_word1)

In [496]:
df1['adj_after_stemming'] = df1.Text.apply(rem_stopwords_stemming).apply(lambda x : ",".join([a for a,b in TextBlob(x).pos_tags if b in ['JJ','JJR','JJS']]))

In [497]:
df1['adj_after_lemmatization'] = df1.Text.apply(rem_stopwords).apply(lemma_spacy).apply(lambda x : ",".join([a for a,b in TextBlob(x).pos_tags if b in ['JJ','JJR','JJS']]))

In [498]:
df1.head(10)

,nouns,ID,Text,wn_sent,Spacy_Entity,textblob_adjtags,textblob_advtags,adjtags_removedSW,adj_after_stemming,adj_after_lemmatization
0,location,2301_1_Q1_1_1,location and friendly professional staff,location and friendly professional staff,[],"friendly,professional",,"friendly,professional",friendli,"friendly,professional"
1,staff,2301_1_Q1_1_2,location and friendly professional staff,location and friendly professional staff,[],"friendly,professional",,"friendly,professional",friendli,"friendly,professional"
2,valet,2301_1_Q1_2_1,valet service was a plus,valet service was a plus,[],,,,,
3,service,2301_1_Q1_2_2,valet service was a plus,valet service was a plus,[],,,,,
4,plus,2301_1_Q1_2_3,valet service was a plus,valet service was a plus,[],,,,,
5,thing,2301_2_Q1_1_1,it was great the only thing that slightly bugg...,it was great the only thing that slightly bugg...,[],"great,only,filled",slightly,great,great,"great,bug"
6,water,2301_2_Q1_1_2,it was great the only thing that slightly bugg...,it was great the only thing that slightly bugg...,[],"great,only,filled",slightly,great,great,"great,bug"
7,container,2301_2_Q1_1_3,it was great the only thing that slightly bugg...,it was great the only thing that slightly bugg...,[],"great,only,filled",slightly,great,great,"great,bug"
8,hall,2301_2_Q1_1_4,it was great the only thing that slightly bugg...,it was great the only thing that slightly bugg...,[],"great,only,filled",slightly,great,great,"great,bug"
9,elevator,2301_2_Q1_1_5,it was great the only thing that slightly bugg...,it was great the only thing that slightly bugg...,[],"great,only,filled",slightly,great,great,"great,bug"


In [499]:
#l2 = 'I am Vasavi and I live in Bengaluru'

In [500]:
#[a for a,b in TextBlob(l2).pos_tags if b in ["NN", "NNP", "NNS", "NNPS"]]

In [501]:
# a1 = ['a','s','d']
# a2 = ['w','e','s']
# a3 = ['a','d','f','s']

In [502]:
# list(set(a1) & set(a2) & set(a3))

### Getting +1/-1, +2/-2, +3/-3

In [503]:
def plus1words(l1):

    l1 = l1.split(" ")

    f = []

    for i, e in enumerate(l1):

        l = []

        length = len(l1)

        if i == 0:

            l.extend(l1[1:2])

        
        elif ((i > 0) & (i < (length - 1))):

            l.extend(l1[i-1:i])
            l.extend(l1[i+1:i+2])


        elif i == (length - 1):

            l.extend(l1[-2:-1])

        # print(f"Words surrounding  {e} are : {l}")

        a = (e,l)

        f.append(a)

    return f

In [504]:
def plus1(Text, nouns):
  for a,b in plus1words(Text):
    if a == nouns:
      return b 

In [505]:
df1["1wordvicinity"] = df1.apply(lambda x: plus1(x['Text'], x['nouns']), axis = 1)
df1.head(3)

,nouns,ID,Text,wn_sent,Spacy_Entity,textblob_adjtags,textblob_advtags,adjtags_removedSW,adj_after_stemming,adj_after_lemmatization,1wordvicinity
0,location,2301_1_Q1_1_1,location and friendly professional staff,location and friendly professional staff,[],"friendly,professional",,"friendly,professional",friendli,"friendly,professional",[and]
1,staff,2301_1_Q1_1_2,location and friendly professional staff,location and friendly professional staff,[],"friendly,professional",,"friendly,professional",friendli,"friendly,professional",[professional]
2,valet,2301_1_Q1_2_1,valet service was a plus,valet service was a plus,[],,,,,,"[, service]"


In [506]:
def plus2words(l1):

    l1 = l1.split(" ")

    f = []

    for i, e in enumerate(l1):

        l = []

        length = len(l1)

        if i == 0:

            l.extend(l1[1:3])

        elif i == 1:

            l.extend(l1[:1])

            l.extend(l1[2:4])

        elif (i > 1 & (i < length - 2)):

            l.extend(l1[i-2:i])

            l.extend(l1[i+1:i+3])

        elif i == len(l1) - 2:

            l.extend(l1[-4:-2])

            l.extend(l1[-1])

        elif i == len(l1) - 1:

            l.extend(l1[-3:])

        # print(f"Words surrounding  {e} are : {l}")

        a = (e,l)

        f.append(a)

    return f

In [507]:
def plus2(Text, nouns):
  for a,b in plus2words(Text):
    if a == nouns:
      return b 

In [508]:
df1["2wordvicinity"] = df1.apply(lambda x: plus2(x['Text'], x['nouns']), axis = 1)
df1.head(3)

,nouns,ID,Text,wn_sent,Spacy_Entity,textblob_adjtags,textblob_advtags,adjtags_removedSW,adj_after_stemming,adj_after_lemmatization,1wordvicinity,2wordvicinity
0,location,2301_1_Q1_1_1,location and friendly professional staff,location and friendly professional staff,[],"friendly,professional",,"friendly,professional",friendli,"friendly,professional",[and],"[and, friendly]"
1,staff,2301_1_Q1_1_2,location and friendly professional staff,location and friendly professional staff,[],"friendly,professional",,"friendly,professional",friendli,"friendly,professional",[professional],"[friendly, professional]"
2,valet,2301_1_Q1_2_1,valet service was a plus,valet service was a plus,[],,,,,,"[, service]","[, service, was]"


In [509]:
def plus3words(l1):

    l1 = l1.split(" ")

    f = []

    for i, e in enumerate(l1):

        l = []

        length = len(l1)

        if i == 0:

            l.extend(l1[1:4])

        elif i == 1:

            l.extend(l1[:1])

            l.extend(l1[2:5])
        elif i == 2:
            l.extend(l1[:2])
            l.extend(l1[3:6])

        elif (i > 2 & (i < length - 3)):

            l.extend(l1[i-3:i])

            l.extend(l1[i+1:i+4])

        elif i == len(l1) - 3:

            l.extend(l1[-6:-3])

            l.extend(l1[-2:])

        elif i == len(l1) - 2:

            l.extend(l1[-5:-2])

            l.extend(l1[-1])

        elif i == len(l1) - 1:

            l.extend(l1[-4:])

        # print(f"Words surrounding  {e} are : {l}")

        a = (e,l)

        f.append(a)

    return f

In [510]:
def plus3(Text, nouns):
  for a,b in plus3words(Text):
    if a == nouns:
      return b 

In [ ]:
df1 = pd.read_excel(r)

In [512]:
df1['noun_phrase'] = df1.apply(lambda x : list(TextBlob(str(x['Text'])).noun_phrases), axis = 1)
df1['relavant_nounphrase'] = df1.apply(lambda x : [i for i in TextBlob(str(x['Text'])).noun_phrases if str(x['nouns']) in i], axis = 1)

In [513]:
df1.head(5)

,nouns,ID,Text,wn_sent,Spacy_Entity,textblob_adjtags,textblob_advtags,adjtags_removedSW,adj_after_stemming,adj_after_lemmatization,1wordvicinity,2wordvicinity,3wordvicinity,noun_phrase,relavant_nounphrase
0,location,2301_1_Q1_1_1,location and friendly professional staff,location and friendly professional staff,[],"friendly,professional",,"friendly,professional",friendli,"friendly,professional",[and],"[and, friendly]","[and, friendly, professional]",[friendly professional staff],[]
1,staff,2301_1_Q1_1_2,location and friendly professional staff,location and friendly professional staff,[],"friendly,professional",,"friendly,professional",friendli,"friendly,professional",[professional],"[friendly, professional]","[and, friendly, professional]",[friendly professional staff],[friendly professional staff]
2,valet,2301_1_Q1_2_1,valet service was a plus,valet service was a plus,[],,,,,,"[, service]","[, service, was]","[, service, was, a]",[valet service],[valet service]
3,service,2301_1_Q1_2_2,valet service was a plus,valet service was a plus,[],,,,,,"[valet, was]","[, valet, was, a]","[, valet, was, a, plus]",[valet service],[valet service]
4,plus,2301_1_Q1_2_3,valet service was a plus,valet service was a plus,[],,,,,,[a],"[was, a]","[service, was, a]",[valet service],[]


### Picking common adjective from 1,2,3 word vicinity and noun phrases

In [514]:
df1.textblob_adjtags = df1.textblob_adjtags.map(str).apply(lambda x: x.split(","))

In [515]:
df1["1wordvicinity"] = df1["1wordvicinity"].fillna(" ")
df1["1wordvicinity"] = df1.apply(lambda x: list(set([i for i in x['textblob_adjtags'] if i in x['1wordvicinity']])), axis = 1)

In [516]:
df1['1wordvicinity'] = df1['1wordvicinity'].apply(lambda x: ",".join(x))

In [517]:
df1["2wordvicinity"] = df1["2wordvicinity"].fillna(" ")
df1["2wordvicinity"] = df1.apply(lambda x: list(set([i for i in x['textblob_adjtags'] if i in x['2wordvicinity']])), axis = 1)

In [518]:
df1['2wordvicinity'] = df1['2wordvicinity'].apply(lambda x: ",".join(x))

In [519]:
df1["3wordvicinity"] = df1["3wordvicinity"].fillna(" ")
df1["3wordvicinity"] = df1.apply(lambda x: list(set([i for i in x['textblob_adjtags'] if i in x['3wordvicinity']])), axis = 1)

In [520]:
df1['3wordvicinity'] = df1['3wordvicinity'].apply(lambda x: ",".join(x))

In [521]:
df1['nounphrase_descriptor'] = df1.apply(lambda x: [i.replace(x['nouns'],"").strip() for i in x['relavant_nounphrase']], axis = 1)

In [522]:
df1['relavant_nounphrase_list'] = df1.relavant_nounphrase.apply(lambda x: (",".join(x)).strip().split())

In [523]:
df1['textblob_adjtags'] = df1.textblob_adjtags.apply(lambda x: [i.strip() for i in x])

In [524]:
df1['nounphrase_adj'] = df1.apply(lambda x: list(set([i for i in x['textblob_adjtags'] if i in x['relavant_nounphrase_list']])), axis = 1)

In [525]:
df1['nounphrase_descriptor'] = df1['nounphrase_descriptor'].apply(lambda x: ",".join(x))

In [526]:
df1['nounphrase_adj'] = df1['nounphrase_adj'].apply(lambda x: ",".join(x))

In [527]:
df1['textblob_adjtags'] = df1['textblob_adjtags'].apply(lambda x: ",".join(x))

In [528]:
df1['noun_phrase'] = df1['noun_phrase'].apply(lambda x: ",".join(x))

In [529]:
df1['relavant_nounphrase'] = df1['relavant_nounphrase'].apply(lambda x: ",".join(x))

In [530]:
df1.head(5)

,nouns,ID,Text,wn_sent,Spacy_Entity,textblob_adjtags,textblob_advtags,adjtags_removedSW,adj_after_stemming,adj_after_lemmatization,1wordvicinity,2wordvicinity,3wordvicinity,noun_phrase,relavant_nounphrase,nounphrase_descriptor,relavant_nounphrase_list,nounphrase_adj
0,location,2301_1_Q1_1_1,location and friendly professional staff,location and friendly professional staff,[],"friendly,professional",,"friendly,professional",friendli,"friendly,professional",,friendly,"professional,friendly",friendly professional staff,,,[],
1,staff,2301_1_Q1_1_2,location and friendly professional staff,location and friendly professional staff,[],"friendly,professional",,"friendly,professional",friendli,"friendly,professional",professional,"professional,friendly","professional,friendly",friendly professional staff,friendly professional staff,friendly professional,"[friendly, professional, staff]","professional,friendly"
2,valet,2301_1_Q1_2_1,valet service was a plus,valet service was a plus,[],,,,,,,,,valet service,valet service,service,"[valet, service]",
3,service,2301_1_Q1_2_2,valet service was a plus,valet service was a plus,[],,,,,,,,,valet service,valet service,valet,"[valet, service]",
4,plus,2301_1_Q1_2_3,valet service was a plus,valet service was a plus,[],,,,,,,,,valet service,,,[],


In [531]:
df1.columns

Index(['nouns', 'ID', 'Text', 'wn_sent', 'Spacy_Entity', 'textblob_adjtags',
       'textblob_advtags', 'adjtags_removedSW', 'adj_after_stemming',
       'adj_after_lemmatization', '1wordvicinity', '2wordvicinity',
       '3wordvicinity', 'noun_phrase', 'relavant_nounphrase',
       'nounphrase_descriptor', 'relavant_nounphrase_list', 'nounphrase_adj'],
      dtype='object')

In [532]:
df1 = df1.reindex(columns = ['ID', 'Text', 'nouns', 'wn_sent', 'Spacy_Entity', 'textblob_adjtags',
       'textblob_advtags', 'adjtags_removedSW', 'adj_after_stemming',
       'adj_after_lemmatization', '1wordvicinity', '2wordvicinity',
       '3wordvicinity', 'noun_phrase', 'relavant_nounphrase',
       'nounphrase_descriptor' , 'nounphrase_adj'])

In [533]:
df1.sample(10)

,ID,Text,nouns,wn_sent,Spacy_Entity,textblob_adjtags,textblob_advtags,adjtags_removedSW,adj_after_stemming,adj_after_lemmatization,1wordvicinity,2wordvicinity,3wordvicinity,noun_phrase,relavant_nounphrase,nounphrase_descriptor,nounphrase_adj
20998,17504_552_Q2_1_1,we arrived at 430 pm and didnt get a room unti...,pm,we arrived at 430 pm and didnt get a room unti...,"[(430 pm, TIME), (8pm, TIME)]",,,nt,,"arrive,nt",,,,,,,
21577,17504_634_Q2_1_1,would have had our room cleaned by 300 pm,room,would have had our room cleaned by 300 pm,"[(300 pm, TIME)]",,,,,,,,,,,,
2358,2301_526_Q1_2_1,,,,[],,,,,,,,,,,,
2029,2301_447_Q1_1_9,"concierge, front desk staff, decor, location, ...",cottage,concierge front desk staff decor location all ...,"[(decor, ORG), (5, CARDINAL)]","front,top,small,modern,large,upscale",,"desk,small,modern,large,upscale","front,top,small,modern,upscal","desk,small,modern,large,upscale",modern,"small,modern","small,modern","front desk staff,star top notch i,small modern...",small modern cottage,small modern,"small,modern"
3511,2301_806_Q1_1_1,i enjoyed being close to the action but also n...,i,i enjoyed being close to the action but also n...,[],"close,able","also,not","close,able,hear","close,hear,anyth","close,able,hear",,,close,hotel room,,,
6532,17504_311_Q1_2_1,loved the well dressed and well spoken guy b...,well,loved the well dressed and well spoken guy beh...,[],checkin,well,"dressed,spoken,checkin","spoken,checkin","spoken,checkin",,,,checkin area,,,
9262,17504_894_Q1_2_1,"the shuttle service is excellent, the drivers...",service,the shuttle service is excellent the drivers w...,[],"shuttle,excellent,excellent",,shuttle,servic,shuttle,shuttle,shuttle,shuttle,shuttle service,shuttle service,shuttle,shuttle
9477,17504_941_Q1_2_1,overall a great experience,experience,overall a great experience,[],"overall,great",,"overall,great","overal,great","overall,great",great,great,"great,overall",great experience,great experience,great,great
16931,17504_19_Q2_1_2,"better tv channels, microwave in the room",channels,better tv channels microwave in the room,[],,better,,,,,,,tv channels,tv channels,tv,
24173,17504_1069_Q2_11_1,really great service,service,really great service,[],great,really,great,great,great,great,great,great,great service,great service,great,great


In [534]:
############################Filtering Using IHG List########################################

In [535]:
dict = pd.read_excel(r"C:\Users\u326421\Desktop\IHG - Text\Entity List.xlsx")

In [536]:
dict["Entity"] = dict["Entity"].str.lower()

In [537]:
ent_list = list(dict["Entity"])

In [538]:
df1["matched_words"] = df1["nouns"].apply(lambda x: list(x for x in x.split() if x in ent_list))

In [539]:
df1["IHG_List_Matching"] = df1.nouns.apply(lambda x: "Yes" if x in ent_list else "No")

In [540]:
#######################################################################################################

In [541]:
#Detecting Not and No from the sentence
df1.apply(lambda x: ",".join([i.strip() for i in x['Text'].split() if i in ['not','no']]), axis = 1)

0         
1         
2         
3         
4         
        ..
24775     
24776     
24777     
24778     
24779     
Length: 24780, dtype: object

In [542]:
#Create a list of not words
w1 = df1['adjtags_removedSW'].map(str).apply(lambda x : x.split(",")).apply(pd.Series).stack().reset_index(name = 'word')

In [1]:
w1['new'] = w1.word.apply(lambda x: 'no' + str(x))
w1['new1'] = w1.word.apply(lambda x: 'not' + str(x))
w1['new2'] = w1.word.apply(lambda x: "couldn't" + str(x))
w1['new3'] = w1.word.apply(lambda x: "shouldn't" + str(x))


NameError: name 'w1' is not defined

In [544]:
desc_words = pd.concat([w1.new,w1.new1,w1.new2,w1.new3], ignore_index = True)

In [545]:
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab)

In [546]:
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
# the list containing the pharses to be matched
terminology_list = desc_words
# convert the phrases into document object using nlp.make_doc to #speed up.
patterns = [nlp.make_doc(text) for text in terminology_list]
# add the patterns to the matcher object without any callbacks
matcher.add("Phrase Matching", None, *patterns)
# the input text string is converted to a Document object

def match_phrase(text):
  doc = nlp(text)
  #call the matcher object the document object and it will return #match_id, start and stop indexes of the matched words
  matches = matcher(doc)
  #print the matched results and extract out the results
  l1 = []
  for match_id, start, end in matches:
      # Get the string representation 
      string_id = nlp.vocab.strings[match_id]  
      span = doc[start:end]  # The matched span
      #print(match_id, string_id, start, end, span.text)
      l1.append(span.text)
  return l1

In [547]:
df1["Not_No_Words"] = df1.Text.apply(lambda x: match_phrase(x))

In [548]:
df1['Not_No_Words'] = df1['Not_No_Words'].apply(lambda x: ", ".join(x))

### Getting Sentiment Scores - VADER

In [549]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [550]:
analyser = SentimentIntensityAnalyzer()

In [551]:
#Getting Vader Sentiment Scores
df1["Vader_Positive"] = df1["Text"].map(str).apply(lambda x: analyser.polarity_scores(x)['pos'])
df1["Vader_Negative"] = df1["Text"].map(str).apply(lambda x: analyser.polarity_scores(x)['neg'])
df1["Vader_Neutral"] = df1["Text"].map(str).apply(lambda x: analyser.polarity_scores(x)['neu'])
df1["Vader_Compound"] = df1["Text"].map(str).apply(lambda x: analyser.polarity_scores(x)['compound'])

In [552]:
def sentimenttags(x):
    if x > 0.00 : 
        return("Positive") 
  
    elif x < 0.00 : 
        return("Negative") 
  
    else : 
        return("Neutral")

In [553]:
df1["Sentiment_Tags"] = df1.Vader_Compound.apply(lambda x: sentimenttags(x))

In [554]:
df1.head(5)

,ID,Text,nouns,wn_sent,Spacy_Entity,textblob_adjtags,textblob_advtags,adjtags_removedSW,adj_after_stemming,adj_after_lemmatization,...,nounphrase_descriptor,nounphrase_adj,matched_words,IHG_List_Matching,Not_No_Words,Vader_Positive,Vader_Negative,Vader_Neutral,Vader_Compound,Sentiment_Tags
0,2301_1_Q1_1_1,location and friendly professional staff,location,location and friendly professional staff,[],"friendly,professional",,"friendly,professional",friendli,"friendly,professional",...,,,[location],Yes,,0.444,0.0,0.556,0.4939,Positive
1,2301_1_Q1_1_2,location and friendly professional staff,staff,location and friendly professional staff,[],"friendly,professional",,"friendly,professional",friendli,"friendly,professional",...,friendly professional,"professional,friendly",[staff],Yes,,0.444,0.0,0.556,0.4939,Positive
2,2301_1_Q1_2_1,valet service was a plus,valet,valet service was a plus,[],,,,,,...,service,,[valet],Yes,,0.000,0.0,1.000,0.0000,Neutral
3,2301_1_Q1_2_2,valet service was a plus,service,valet service was a plus,[],,,,,,...,valet,,[service],Yes,,0.000,0.0,1.000,0.0000,Neutral
4,2301_1_Q1_2_3,valet service was a plus,plus,valet service was a plus,[],,,,,,...,,,[],No,,0.000,0.0,1.000,0.0000,Neutral


In [555]:
##############################################################################################

In [ ]:
dd1 = df1

In [ ]:
#Finding unique adjectives from multiple columns
dd1['common'] = dd1['textblob_adjtags'].fillna(" ") + "," + dd1['adjtags_removedSW'].fillna(" ") + "," + dd1['adj_after_stemming'].fillna(" ") + "," + dd1['adj_after_lemmatization'].fillna(" ") + "," + dd1['1wordvicinity'].fillna(" ")\
 + "," + dd1['2wordvicinity'].fillna(" ") + "," + dd1['3wordvicinity'].fillna(" ")

dd1['common'] = dd1['common'].apply(lambda x: list(set(x.split(","))))
dd1['common'].head()

In [ ]:
#################Creating 2 or more word entities from noun-phrases#####################################

In [ ]:
dd1['common1'] = dd1['common'].map(str)
t1 = dd1[['ID','noun_phrase','common1']].set_index(['ID','common1']).noun_phrase.apply(lambda x: str(x).split(",")).apply(pd.Series).stack().reset_index(name = 'np')
t1.head()

In [ ]:
t1.common1 = t1.common1.apply(lambda x: x.replace("[","")).\
apply(lambda x: x.replace("]","")).\
apply(lambda x: x.replace("'","")).apply(lambda x: x.split(","))

In [ ]:
t1.common1 = t1.common1.apply(lambda x: [i.strip() for i in x])

In [ ]:
t1['np_entity'] = t1.apply(lambda x: " ".join([i for i in x['np'].split(" ") if i not in x['common1']]), axis = 1)
t2 = t1.groupby('ID').np_entity.apply(list).reset_index()

In [ ]:
t2.np_entity = t2.np_entity.apply(lambda x: ",".join(x))

In [ ]:
dd1 = dd1.merge(t2, left_on = "ID", how = "left")

In [ ]:
########################Getting final descriptor from multiple columns##########################

for i in range(dd1.shape[0]):
    
    if (~(dd1['1wordvicinity'].isna().values[i])):
        dd1.loc[i,'new_col'] = dd1['1wordvicinity'][i]
        
    elif ~(dd1['2wordvicinity'].isna().values[i]):
        dd1.loc[i,'new_col'] = dd1['2wordvicinity'][i]
        
    elif ~(df1['3wordvicinity'].isna().values[i]):
        dd1.loc[i,'new_col'] = dd1['3wordvicinity'][i]
        
    elif ~(dd1['nounphrase_adj'].isna().values[i]):
        dd1.loc[i,'new_col'] = dd1['nounphrase_adj'][i]
        
    elif ~(dd1['textblob_adjtags'].isna().values[i]):
        dd1.loc[i,'new_col'] = dd1['textblob_adjtags'][i]
        
    else:
        dd1.loc[i,'new_col'] = 'No Descriptor Found'
    

In [ ]:
dd = dd1[['ID', 'Entity', 'np_entity']]

In [ ]:
dd.ID = dd.ID.apply(lambda x: x[0:-2])

In [ ]:
dd = dd.set_index('ID').stack()

In [556]:
#df1.to_excel(r"C:\Users\u326421\Desktop\IHG - Text\IHG_Output.xlsx")